In [218]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import plotly as plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff

%matplotlib inline
import matplotlib.pyplot as plt
from scipy import ndimage
import seaborn as sns

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
plotly.tools.set_credentials_file(username='itjallingii', api_key='hiFqLEhfwSwdDlsj9lT8')

#sources for plotting
#https://seaborn.pydata.org/tutorial/categorical.html
#https://flowingdata.com/2019/03/06/women-men-timeuse/

#To DO FROM NOW
#1. resolve error in summing
#2. combine delay time with traffic data --> laura script
#3. find a way to visualise the characteristics of the 10 most vulnerable/critical bridges with plotly or see seaborn thing

In [228]:
#GOAL OF THIS SCRIPT: synchronise LRP road data with segment label (link traffic data to bridges)
df_all_n1["Road_segment_no"] =""
df_all_n1["Road_segment_label"] =""

#iterate through all road segments and get values for that segment
for row_traffic in range(len(df_n1_traffic)):
    chainage_begin_of_segment = df_n1_traffic.Start_Chainage.iloc[row_traffic,]
    chainage_end_of_segment = df_n1_traffic.End_Chainage.iloc[row_traffic,]
    Road_segment_label = df_n1_traffic.Link_Name.iloc[row_traffic,]
    Road_segment_no = df_n1_traffic.Link_No.iloc[row_traffic,]

#iterate through all LRP points (rows) and set values for that segment
    for row_road in range(len(df_all_n1)):
        if df_all_n1.Chainage.iloc[row_road,] >= chainage_begin_of_segment:
            if df_all_n1.Chainage.iloc[row_road,] < chainage_end_of_segment:        
                df_all_n1.Road_segment_label.loc[row_road,] = Road_segment_label
                df_all_n1.Road_segment_no.loc[row_road,] = Road_segment_no
        else:
            pass

In [229]:
#GOAL OF THIS SCRIPT: Get number of A bridges/B Bridges/C/D bridges of road data to road segments
#iterate through all road segments and get values for that segment
road_list = ["N1"]
index = 0

df_n1_traffic["Total_A"] = ""
df_n1_traffic["Total_B"] = ""
df_n1_traffic["Total_C"] = ""
df_n1_traffic["Total_D"] = ""
df_n1_traffic["segment_vulnerability"] = ""
df_n1_traffic["segment_vulnerability_length"] = ""

for road in road_list:
    #load in all the roads
    df_road = df_all_n1
    segment_list = df_all_n1["Road_segment_no"].unique()
    segment_list = segment_list.tolist()

    for segments in segment_list:
        #make small dataframe to count bridges
        df_segment = df_road[df_road["Road_segment_no"] == segments]
        Bridge_count = df_segment.groupby(["Cat"]).count()
        #number_types = Bridge_count.Road.count()
        Bridge_count = Bridge_count["Road"]
        categories = Bridge_count.index.values.tolist()
        #number_types = Bridge_count.Road.count()
        
        #count number of bridges per category and check if they are they are there
        if 'A' in categories:
            total_A_Bridges = Bridge_count['A']
        else:
            total_A_Bridges = 0
        if 'B' in categories:
            total_B_Bridges = Bridge_count['B']
        else:
            total_B_Bridges = 0
        if 'C' in categories:
            total_C_Bridges = Bridge_count['C']
        else:
            total_C_Bridges = 0
        if 'D' in categories:
            total_D_Bridges = Bridge_count['D']
        else:
            total_D_Bridges = 0 
            
        segment_vulnerability = total_A_Bridges * scenario_df.iloc[7]['A_overall'] + total_B_Bridges * scenario_df.iloc[7]['B_overall'] + total_C_Bridges * scenario_df.iloc[7]['C_overall'] + total_D_Bridges * scenario_df.iloc[7]['D_overall'] 
        
        #store total bridges in segment traffic file       
        df_n1_traffic.Total_A.loc[index,] =  total_A_Bridges
        df_n1_traffic.Total_B.loc[index,] =  total_B_Bridges
        df_n1_traffic.Total_C.loc[index,] =  total_C_Bridges
        df_n1_traffic.Total_D.loc[index,] =  total_D_Bridges
        df_n1_traffic.segment_vulnerability.loc[index,] = segment_vulnerability
        #df_n1_traffic.segment_vulnerability_length.loc[index,] = df_n1_traffic.segment_vulnerability.iloc[index,] / df_n1_traffic.Link_Length.iloc[index,]
        
        index = index + 1

In [225]:
# Create a list of all the LRP's of the bridges
df_all = pd.read_csv('data_clean_all.csv')
df_n1 = pd.read_csv('Data_clean_N1_Dhaka_Chittagong.csv')
df_all["average_delay"] = np.nan
df_n1["average_delay"] = np.nan
df_n1_traffic = pd.read_csv('N1_data_traffic.csv')

#add expected delay based on length and CAT per bridge --> 
#delaytime per bridge is delaytime(length) * prob_breakingdown(Cat)
#chose different scenario's --> Chose bridges breaking down? 

#traffic data is in ADTT (the total volume of vehicle traffic in both directions on a section of road for a year divided by 365 days)
# Still, the AADT gives a very good indication about the importance of a road, and about the number of truck passages per day or per year, which can be translated to the number of tonnes transported via that road per year.
# ividing the AADT by the number of lanes provides insight into how busy each lane is, and where we can expect most congestion
d = df_all[df_all["Cat"] == "A"]
k = d[d["Road"] == "N1"]
df_all_n1 = df_all[df_all["Road"] == "N1"]

df_n1_traffic = df_n1_traffic.rename(columns={'End Chainage': 'End_Chainage', 'No Lanes': 'No_Lanes','Start Chainage': 'Start_Chainage','Link no': 'Link_No','Link Name':'Link_Name',"Link Length": "Link_Length"})

In [226]:
d = {'Road': ["N1", "N4","Z808","N87","N33" ], 'Criticality': [4302, 3500,3700,2400,2600], "Vulnerability": [30,35,47,38,51]}
rand_df = pd.DataFrame(data=d)
road_roadnames = pd.read_csv('_roadnames_list.csv')
road_roadnames_list = list(road_roadnames.columns.values)
df_n1_traffic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 29 columns):
Unnamed: 0             69 non-null int64
Road name              69 non-null object
Link_No                69 non-null object
Link_Name              69 non-null object
Start LRP              69 non-null object
Start Offset           69 non-null int64
Start_Chainage         69 non-null float64
End LRP                69 non-null object
End Offset             69 non-null int64
End_Chainage           69 non-null float64
Link_Length            69 non-null float64
Economic Traffic       69 non-null float64
No_Lanes               69 non-null int64
Heavy Truck            69 non-null float64
Medium Truck           69 non-null float64
Small Truck            69 non-null float64
Large Bus              69 non-null float64
Medium Bus             69 non-null float64
Micro Bus              69 non-null float64
Utility                69 non-null float64
Car                    69 non-null float64
Auto Ri

In [227]:
#GOAL OF THIS SCRIPT: Determine vulnerability of bridges based on their scenario likelihood
scenario = {'Scenario': [1,2,3,4,5,6,7,8], 'A': [0,0,0,0,0,0,0.05,0.10], 
            'B': [0,0,0,0,0.05,0.05,0.10,0.20],'C':[0,0,0,0.10,0.10,0.20,0.20,0.40],'D':[0,0.05,0.10,0.20,0.20,0.40,0.40,0.80],
            'Scenario_likelihood': [0,0.40,0.30,0.10,0.075,0.050,0.05,0.025]}
scenario_df = pd.DataFrame(data=scenario)
scenario_df = scenario_df.drop(scenario_df.index[0])

scenario_df["A_overall"] = scenario_df.Scenario_likelihood *scenario_df.A
scenario_df["B_overall"] = scenario_df.Scenario_likelihood *scenario_df.B
scenario_df["C_overall"] = scenario_df.Scenario_likelihood *scenario_df.C
scenario_df["D_overall"] = scenario_df.Scenario_likelihood *scenario_df.D

sum_row = {col: scenario_df[col].sum() for col in scenario_df}
scenario_df_sum = pd.DataFrame(sum_row, index=["Total"])
scenario_df = scenario_df.append(scenario_df_sum)
#---------> A_overall total is vulnerability of bridge 

In [202]:
#setup dataframe to store aggregate data
statistics = {'road_name':[""],'road_vulnerability':[""],'segment_name':[""],'number_A_bridges':[""], 'number_B_bridges':[""],'number_C_bridges':[""],'number_D_bridges':[""],
              'average_bridge_length':[""],'road_length':[""],'average_delay_bridges': [""]}
descriptive_df = pd.DataFrame(data=statistics,index = range(0,885))
#-----> check dimensions of segment data first before starting handling the data

In [217]:
descriptive_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 885 entries, 0 to 884
Data columns (total 10 columns):
road_name                885 non-null object
road_vulnerability       885 non-null object
segment_name             885 non-null object
number_A_bridges         885 non-null object
number_B_bridges         885 non-null object
number_C_bridges         885 non-null object
number_D_bridges         885 non-null object
average_bridge_length    885 non-null object
road_length              885 non-null object
average_delay_bridges    885 non-null object
dtypes: object(10)
memory usage: 69.2+ KB


In [219]:
df_all_n1

,Unnamed: 0,Road,LRPName,Lat,Lon,Chainage,RLRPN,Description,Length,Cat,Next Node,Add On Process,average_delay,Road_segment_no,Road_segment_label
0,0,N1,LRPS,23.706028,90.443333,0.000,N1LRPS,0,0.0,0,N1LRPSa,BlankProcess,NaN,N1-1,Jatrabari - Int.with Z1101 (Left) (Left)
1,1,N1,LRPSa,23.702917,90.450417,0.814,N1LRPSa,0,0.0,0,N1LRPSb,BlankProcess,NaN,N1-1,Jatrabari - Int.with Z1101 (Left) (Left)
2,2,N1,LRPSb,23.702778,90.450472,0.822,N1LRPSb,0,0.0,0,N1LRP001,BlankProcess,NaN,N1-2R,Int.with Z1101 - Signboard (Left) R111 (Right)
3,3,N1,LRP001,23.702139,90.451972,1.000,N1LRP001,0,0.0,0,N1LRP001a,BlankProcess,NaN,N1-2R,Int.with Z1101 - Signboard (Left) R111 (Right)
4,4,N1,LRP001a,23.698739,90.458861,1.800,N1LRP001a,.,11.3,A,N1LRP002,BridgesDelay,0.005625,N1-2R,Int.with Z1101 - Signboard (Left) R111 (Right)
5,5,N1,LRP002,23.697889,90.460583,2.000,N1LRP002,0,0.0,0,N1LRP002a,BlankProcess,NaN,N1-2R,Int.with Z1101 - Signboard (Left) R111 (Right)
6,6,N1,LRP002a,23.697361,90.461667,2.130,N1LRP002a,0,0.0,0,N1LRP003,BlankProcess,NaN,N1-2R,Int.with Z1101 - Signboard (Left) R111 (Right)
7,7,N1,LRP003,23.693833,90.469138,3.000,N1LRP003,0,0.0,0,N1LRP004,BlankProcess,NaN,N1-2R,Int.with Z1101 - Signboard (Left) R111 (Right)
8,8,N1,LRP004,23.693611,90.478777,4.000,N1LRP004,0,0.0,0,N1LRP004a,BlankProcess,NaN,N1-2R,Int.with Z1101 - Signboard (Left) R111 (Right)
9,9,N1,LRP004a,23.693805,90.480527,4.175,N1LRP004a,0,0.0,0,N1LRP004b,BlankProcess,NaN,N1-3,Signboard - Shimrail (Left)R110 (Left)


In [ ]:
#goal of this script: calculate delay time caused by each road and store vulnerability in aggregated file
index = 0
road_list = ["N1","N202"]
for i in road_list:
    #load in all the roads
    df_selected = df_all
    print(descriptive_df)
    
    #determine delaytime per bridge
    for row in range(len(df_selected)):
        if df_selected.Cat.iloc[row,] != 0:        
            if df_selected.Length.iloc[row,] < 10:
                if df_selected.Cat.iloc[row,] == "A": 
                    df_selected.average_delay.iloc[row,] = 15 / 60 * scenario_df.iloc[7]['A_overall']                        
                if df_selected.Cat.iloc[row,] == "B": 
                    df_selected.average_delay.iloc[row,] = 15 / 60 * scenario_df.iloc[7]['B_overall']
                if df_selected.Cat.iloc[row,] == "C":
                    df_selected.average_delay.iloc[row,] = 15 / 60 * scenario_df.iloc[7]['C_overall']
                if df_selected.Cat.iloc[row,] == "D":
                    df_selected.average_delay.iloc[row,] = 15 / 60 * scenario_df.iloc[7]['D_overall']
            if df_selected.Length.iloc[row,] < 50:
                if df_selected.Cat.iloc[row,] == "A": 
                    df_selected.average_delay.iloc[row,] = 37.5 / 60 * scenario_df.iloc[7]['A_overall']
                if df_selected.Cat.iloc[row,] == "B": 
                    df_selected.average_delay.iloc[row,] = 37.5 / 60 * scenario_df.iloc[7]['B_overall']
                if df_selected.Cat.iloc[row,] == "C":
                    df_selected.average_delay.iloc[row,] = 37.5 / 60 * scenario_df.iloc[7]['C_overall']
                if df_selected.Cat.iloc[row,] == "D":
                    df_selected.average_delay.iloc[row,] = 37.5 / 60 * scenario_df.iloc[7]['D_overall']   
            if df_selected.Length.iloc[row,] < 200:
                if df_selected.Cat.iloc[row,] == "A": 
                    df_selected.average_delay.iloc[row,] = 67.5 / 60  * scenario_df.iloc[7]['A_overall']
                if df_selected.Cat.iloc[row,] == "B": 
                    df_selected.average_delay.iloc[row,] = 67.5 / 60 * scenario_df.iloc[7]['B_overall']
                if df_selected.Cat.iloc[row,] == "C":
                    df_selected.average_delay.iloc[row,] = 67.5 / 60 * scenario_df.iloc[7]['C_overall']
                if df_selected.Cat.iloc[row,] == "D":
                    df_selected.average_delay.iloc[row,] = 67.5 / 60 * scenario_df.iloc[7]['D_overall'] 
            if df_selected.Length.iloc[row,] > 200:
                if df_selected.Cat.iloc[row,] == "A": 
                    df_selected.average_delay.iloc[row,] = 2.333 * scenario_df.iloc[7]['A_overall']
                if df_selected.Cat.iloc[row,] == "B": 
                    df_selected.average_delay.iloc[row,] = 2.333 * scenario_df.iloc[7]['B_overall']
                if df_selected.Cat.iloc[row,] == "C":
                    df_selected.average_delay.iloc[row,] = 2.333 * scenario_df.iloc[7]['C_overall']
                if df_selected.Cat.iloc[row,] == "D":
                    df_selected.average_delay.iloc[row,] = 2.333 * scenario_df.iloc[7]['D_overall'] 
            #we need to append here with multiple ROADS !!!!
            df_all_n1 = df_selected
            
            #goal of this script --> save data in aggregated file
    #save descriptive data
    descriptive_df.road_name.iloc[index,] = i
        
    #make small dataframe to count bridges
    bridge_count = df_segment.groupby(["Cat"]).count()
        
    #number_types = Bridge_count.Road.count()
    bridge_count = bridge_count["Road"]
    categories = Bridge_count.index.values.tolist()
        
    #count number of bridges per category and check if they are they are there
    if 'A' in categories:     
        descriptive_df.number_A_bridges.loc[index,] = Bridge_count['A']
    else:
        descriptive_df.number_A_bridges.loc[index,] = 0
    if 'B' in categories:
        descriptive_df.number_B_bridges.loc[index,] = Bridge_count['B']
    else:
        descriptive_df.number_B_bridges.loc[index,] = 0
    if 'C' in categories:
        descriptive_df.number_C_bridges.loc[index,] = Bridge_count['C']
    else:
        descriptive_df.number_C_bridges.loc[index,] = 0
    if 'D' in categories:
        descriptive_df.number_D_bridges.loc[index,] = Bridge_count['D']
    else:
        descriptive_df.number_D_bridges.loc[index,] = 0 
        
    #store vulnerability in overall aggrageted dataframe
    descriptive_df.road_vulnerability.iloc[index,] = descriptive_df.number_A_bridges.loc[index,] * scenario_df.iloc[7]['A_overall'] + descriptive_df.number_B_bridges.loc[index,] * scenario_df.iloc[7]['B_overall'] + descriptive_df.number_C_bridges.loc[index,] * scenario_df.iloc[7]['C_overall'] + descriptive_df.number_D_bridges.loc[index,] * scenario_df.iloc[7]['D_overall']      
        
    #determine average bridge lenght and road length
    length_df = df_selected[df_selected["Cat"] != "0"]
    length = length_df["Cat"].count()
    length_1 = df_selected["Cat"].count() - 1    
    
    #get delaytime from all the bridges
    descriptive_df.average_delay_bridges.iloc[index,] = df_selected.average_delay.sum()
    descriptive_df.average_bridge_length.iloc[index,] = length_df.Length.sum() / length
 
    #get roadlength based on chainage from all the bridges
    descriptive_df.road_length.iloc[index,] = df_selected.Chainage.iloc[length_1,]
    #update index to iterate over alll the roads
    index = index + 1 

    
    
    print(descriptive_df)

,road_name,road_vulnerability,segment_name,number_A_bridges,number_B_bridges,number_C_bridges,number_D_bridges,average_bridge_length,road_length,average_delay_bridges
0,,,,,,,,,,
1,,,,,,,,,,
2,,,,,,,,,,
3,,,,,,,,,,
4,,,,,,,,,,
5,,,,,,,,,,
6,,,,,,,,,,
7,,,,,,,,,,
8,,,,,,,,,,
9,,,,,,,,,,


In [15]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Bar(
    x=['giraffes', 'orangutans', 'monkeys'],
    y=[20, 14, 23],
    name='SF Zoo'
)
trace2 = go.Bar(
    x=['giraffes', 'orangutans', 'monkeys'],
    y=[12, 18, 29],
    name='LA Zoo'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig) 

In [ ]:
#DENSITY PLOTS
#density here sorted
grades=sorted(df.grade.unique())
data=[]
for g in grades:
    #criticality here
    df_grade=df[df.grade==g]
    data.append(
        go.Scattergl(
            x=df_grade.sqft_living15,
            y=np.log(df_grade.price),
            mode='markers',
            text=[f'Living Room Area:{df_grade.at[i, "sqft_living15"]} sq.ft.<br>Grade:{df_grade.at[i, "grade"]}<br>Price:${df_grade.at[i, "price"]}' for i in df_grade.index],
            marker=dict(
                opacity=0.75,
            ),
            name='Grade:'+str(g)
        )
    )

figure = go.Figure(data=data, layout=layout)

ply.iplot(figure)  

In [ ]:
#CHARACTERISTIC CHARTS about top 10 segments and roads
stunting = "SH.STA.STNT.ZS"
wasting = "SH.STA.WAST.ZS"
overweight = "SH.STA.OWGH.ZS"
anemia = "SH.ANM.ALLW.ZS"
exclusive_breastfeeding = "SH.STA.BFED.ZS"
#Add all the indicators in a list
li = [["SH.STA.STNT.ZS"],["SH.STA.WAST.ZS"],["SH.STA.OWGH.ZS"],["SH.ANM.ALLW.ZS"],["SH.STA.BFED.ZS"]]
indi = []
#Get the data in the right format for all the indicators via a for-loop
for i in range(0,5):
    WDI_SSA_I = WDI_SSA[WDI_SSA["Indicator Code"].isin(li[i])]
    #get all the necessary columns
    SSA_10_17 = pd.concat([WDI_SSA_I.loc[:,'Country Name':'Indicator Code'],WDI_SSA_I.loc[:,'2010':'2017']], axis=1)
    #melt the data into a long format
    melted = pd.melt(SSA_10_17, id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"], var_name="Year", value_name="Value")
    #plot indicator over time for one country
    ll = [ 'Sub-Saharan Africa','World']
    #get only the two regions above and sort them
    mc = melted.loc[melted['Country Name'].isin(ll)] 
    mc = mc.sort_values(["Country Name", "Year"])
    #create two traces, for each region 1
    trace1 = go.Bar(
        x = mc[mc["Country Name"] == 'World']["Year"],
        y = mc[mc["Country Name"] == 'World']["Value"],
        name = 'World',
        legendgroup = 'a',
        showlegend = False,
        marker=dict(color=pblue), 
    )
    trace2 = go.Bar(
        x = mc[mc["Country Name"] == 'Sub-Saharan Africa']["Year"],
        y = mc[mc["Country Name"] == 'Sub-Saharan Africa']["Value"],
        name = 'Sub-Saharan Africa',
        legendgroup = 'b',
        showlegend = False,
        marker=dict(color=porange), 
    )
#Add the two traces together and add them to indi
    data = [trace1, trace2]   
    indi.append(data)
#Plot all the bar charts 
fig = tools.make_subplots(rows=1, cols=5, subplot_titles=('Stunting', 'Wasting', 'Overweight', 'Anemia', 'Exclusive Breastfeeding'), print_grid = False)
for i in range(0,5):
    fig.append_trace(indi[i][0], 1, i+1)
    fig.append_trace(indi[i][1], 1, i+1)
fig['layout'].update(height=300, width=900, title='Figure 4 <br>Target 2.2: Indicators for malnutrition from 2010 till 2017 <br> ')
fig['layout']['yaxis1'].update(title='percentage (%)')
py.iplot(fig, filename='malnutrition-indicators')